## Install dependencies (安装依赖项)

In [ ]:
%pip install -r requirements.txt
%pip install libresvip --no-deps

## Set display language to Chinese (Optional)
## 设置显示语言为中文 (可选)

In [ ]:
from libresvip.core.config import Language, settings
from libresvip.extension.manager import get_translation
from libresvip.utils import translation

settings.language = Language.CHINESE

translation.singleton_translation = get_translation()

In [ ]:
import enum
from typing import get_args, get_type_hints

from pydantic import BaseModel
from pydantic_core import PydanticUndefined
from pydantic_extra_types.color import Color

from libresvip.model.base import BaseComplexModel
from libresvip.utils.translation import gettext_lazy as _

def generate_model_json_schema(option_class: BaseModel, description: str = "") -> dict:
    schema = {
        "type": "object",
        "$defs": {},
        "title": option_class.__name__,
        "properties": {},
        "required": []
    }
    if description:
        schema["description"] = _(description)
    for option_key, field_info in option_class.model_fields.items():
        schema["required"].append(option_key)
        default_value = None if field_info.default is PydanticUndefined else field_info.default
        field_kwargs = {"title": _(field_info.title)}
        if field_info.description is not None:
            field_kwargs["description"] = _(field_info.description)
        if issubclass(field_info.annotation, bool):
            schema["properties"][option_key] = {
                "type": "boolean",
                "x-display": "switch",
                "default": default_value,
                **field_kwargs
            }
        elif issubclass(field_info.annotation, enum.Enum):
            if default_value is not None:
                default_value = str(default_value.value)
            annotations = get_type_hints(field_info.annotation, include_extras=True)
            choices = []
            for enum_item in field_info.annotation:
                if enum_item.name in annotations:
                    annotated_args = list(get_args(annotations[enum_item.name]))
                    if len(annotated_args) >= 2:
                        arg_0, enum_field = annotated_args[:2]
                    else:
                        continue
                    choice = {
                        "const": enum_item.value,
                        "title": _(enum_field.title)
                    }
                    choices.append(choice)
            schema["properties"][option_key] = {
                "type": "string",
                "default": default_value,
                "oneOf": choices,
                **field_kwargs
            }
        elif issubclass(field_info.annotation, int):
            schema["properties"][option_key] = {
                "type": "integer",
                "default": default_value,
                **field_kwargs
            }
        elif issubclass(field_info.annotation, float):
            schema["properties"][option_key] = {
                "type": "number",
                "default": default_value,
                **field_kwargs
            }
        else:
            if issubclass(field_info.annotation, Color):
                field_kwargs["x-display"] = "color-picker"
            elif issubclass(field_info.annotation, BaseComplexModel):
                default_value = field_info.annotation.default_repr()
            schema["properties"][option_key] = {
                "type": "string",
                "default": default_value,
                **field_kwargs
            }
    return schema

In [ ]:
import ipyvuetify as v
import reacton
import reacton.ipyvuetify as rv
import reacton.ipywidgets as w
import traitlets

class TryVjsf(v.VuetifyTemplate):
    template_file = "vjsf.vue"

    vjsf_loaded = traitlets.Bool(False).tag(sync=True)
    form_data = traitlets.Dict(default_value={}).tag(sync=True)
    schema = traitlets.Dict().tag(sync=True)
    valid = traitlets.Bool(False).tag(sync=True)

In [ ]:
from typing import Literal

from pydantic import BaseModel, Field

from libresvip.extension.manager import plugin_manager


class FileFormats(BaseModel):
    __doc__ = _("""Choose file format""")
    input_format: Literal[*plugin_manager.plugin_registry] = Field(title=_("Import format"))
    output_format: Literal[*plugin_manager.plugin_registry] = Field(title=_("Export format"))

file_formats_form = TryVjsf(schema=FileFormats.model_json_schema())
file_formats_form

In [ ]:
from libresvip.extension.manager import middleware_manager

if not file_formats_form.valid:
    raise ValueError


class MiddlewareOptions(BaseModel):
    enabled_middlewares: list[Literal[*(_(middleware_info.name) for middleware_info in middleware_manager.plugin_registry.values())]] = Field(
        default_factory=list, title=_("Enabled middlewares")
    )

input_options_cls = get_type_hints(plugin_manager.plugin_registry[file_formats_form.form_data["input_format"]].plugin_object.load)["options"]
input_options_form = TryVjsf(schema=generate_model_json_schema(input_options_cls))
input_options_header = v.ExpansionPanelHeader(children=[_("Input Options")])
input_options_content = v.ExpansionPanelContent(children=[input_options_form])
input_options_panel = v.ExpansionPanel(children=[input_options_header, input_options_content])

middleware_options_schema = MiddlewareOptions.model_json_schema()
middleware_options_schema.setdefault("required", ["enabled_middlewares"])
for middleware_id, middleware_info in middleware_manager.plugin_registry.items():
    option_schema = generate_model_json_schema(get_type_hints(middleware_info.plugin_object.process)["options"])
    for option_property in option_schema["properties"].values():
        option_property["title"] = _(middleware_info.name) + " - " + option_property["title"]
    middleware_options_schema["properties"].update(option_schema["properties"])
    middleware_options_schema["required"].extend(option_schema["required"])

middleware_options_form = TryVjsf(schema=middleware_options_schema)
middleware_options_header = v.ExpansionPanelHeader(children=[_("""Intermediate Processing""")])
middleware_options_content = v.ExpansionPanelContent(children=[middleware_options_form])
middleware_options_panel = v.ExpansionPanel(children=[middleware_options_header, middleware_options_content])

output_options_cls = get_type_hints(plugin_manager.plugin_registry[file_formats_form.form_data["output_format"]].plugin_object.dump)["options"]
output_options_form = TryVjsf(schema=generate_model_json_schema(output_options_cls))
output_options_header = v.ExpansionPanelHeader(children=[_("Output Options")])
output_options_content = v.ExpansionPanelContent(children=[output_options_form])
output_options_panel = v.ExpansionPanel(children=[output_options_header, output_options_content])

option_panels = v.ExpansionPanels(children=[input_options_panel, middleware_options_panel, output_options_panel])
option_panels

In [ ]:
select_file_schema = {
    'type': 'object',
    'properties': {
        'input_file': {
            'type': 'object',
            'title': 'Select project file',
            'contentMediaType': '*/*',
            'properties': {
                'name': {'type': 'string'},
                'size': {'type': 'number'},
                'type': {'type': 'string'},
                'data': {'type': 'string'},
                'lastModified': {
                    'type': 'string', 'format': 'date-time'
                }
            },
            'writeOnly': True
        }
    }
}
select_file_form = TryVjsf(schema=select_file_schema)
select_file_form

In [ ]:
@reacton.component
def save(data: bytes, file_name: str) -> str:
    try:
        import js
        import pyodide.ffi

        js_data = pyodide.ffi.to_js(data)
        js_blob_obj = js.Blob.new([js_data])
        href = js.URL.createObjectURL(js_blob_obj)
    except ImportError:
        encoded = base64.b64encode(data).decode('utf-8')
        href = f"data:application/octet-stream;base64,{encoded}"
    return rv.Html(
        tag='a',
        attributes={'href': href, "download": file_name},
        children=[f'Download {file_name}']
    )

In [ ]:
import base64
import pathlib
import tempfile

temp_path = pathlib.Path(tempfile.mkdtemp())

input_file_data = select_file_form.form_data["input_file"]

input_plugin = plugin_manager.plugin_registry[file_formats_form.form_data["input_format"]]
output_plugin = plugin_manager.plugin_registry[file_formats_form.form_data["output_format"]]
if (input_option_cls := get_type_hints(input_plugin.plugin_object.load).get("options", None)):
    input_option = input_option_cls.model_validate(input_options_form.form_data)
else:
    raise
if (output_option_cls := get_type_hints(output_plugin.plugin_object.dump).get("options", None)):
    output_option = output_option_cls.model_validate(output_options_form.form_data)
else:
    raise
has_error = False
cur_dir = pathlib.Path(".")
child_file = (temp_path / input_file_data["name"])
child_file.write_bytes(base64.b64decode(input_file_data["data"]))
target_file = (cur_dir / input_file_data["name"]).with_suffix(f".{file_formats_form.form_data['output_format']}")
project = input_plugin.plugin_object.load(child_file, input_option)
for middleware in middleware_manager.plugin_registry.values():
    if _(middleware.name) in middleware_options_form.form_data["enabled_middlewares"]:
        if middleware_option_cls := get_type_hints(middleware.plugin_object.process).get("options"):
            middleware_option = middleware_option_cls.model_validate(middleware_options_form.form_data)
            project = middleware.process(project, middleware_option)
output_plugin.plugin_object.dump(target_file, project, output_option)
save(target_file.read_bytes(), target_file.name)

In [ ]:
@reacton.component
def file_upload():
    items, set_items = reacton.use_state([])

    def on_value(values: tuple[dict]) -> None:
        new_items = []
        for value in values:
            file_name = value["name"]
            new_items.append(file_name)
            pathlib.Path(file_name).write_bytes(value["content"].tobytes())
        set_items(new_items)

    with w.VBox() as main:
        w.FileUpload(
            accept='',
            multiple=True,
            on_value=on_value
        )
        for item in items:
            w.Label(value=item)
    return main

file_upload()